## Peer-graded Assignment week 3: Segmenting and Clustering Neighborhoods in Toronto

This notebook is used for assignment week 3, Datascience Capstone.

### Step 1: Create notebook
Start by creating a new Notebook for this assignment. Check! :)

### Step 2: Web scrape data
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table=soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
#print(df)

In [3]:
# Just double-checking that I have everything
df.describe()
#df.head(30)
#df.tail(30)

,Postcode,Borough,Neighbourhood
count,288,288,288
unique,180,12,209
top,M9V,Not assigned,Not assigned
freq,8,77,78


### Step 3: Create dataframe
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
#Just checking how many rows will disappear
(df['Borough']=='Not assigned').sum()

77

In [5]:
# Drop rows with "Not assigned" in column "Borough"
df1=df[df.Borough != 'Not assigned']

In [9]:
# Double-checking that everything looks good so far
(df1['Borough']=='Not assigned').sum()
df1.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [24]:
df2=df1.groupby(["Postcode","Borough"])['Neighbourhood'].apply(','.join).reset_index()
df2.head(100)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.**

Double-check that there are no cases where neighbourhood is assigned while borough is not assigned. Note that this control is done using the original data set (called 'df' in this assignment), making it possible to identify errors before any modifications.

In [21]:
df[(df['Borough']=='Not assigned') & (df['Neighbourhood']!='Not assigned')]

,Postcode,Borough,Neighbourhood


Identifying rows where neighbourhood needs to be updated:

In [26]:
df2[(df2['Borough']!='Not assigned') & (df2['Neighbourhood']=='Not assigned')]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.